In [20]:
import sys
import os
sys.path.append("address_score")

#### Address Score Calculation

In [21]:
# Get the Greater Vancouver network map cache
from network_cache import cache_greater_vancouver_network

graph_path = "greater_vancouver.graphml"

if not os.path.exists(graph_path):
    print("Graph file not found. Generating it now...")
    cache_greater_vancouver_network(graph_path)
else:
    print("Graph file already exists. Skipping download.")

Graph file already exists. Skipping download.


In [22]:
# Calculating the road distances from desired addresses to houses
from compute_distances import compute_road_distances
# Desired addresses for scoring. This example uses the location of Northeastern University's Vancouver campus.
start_coords = (49.28069261104817, -123.11572527879807)
house_with_distances = compute_road_distances(start_coords,house_csv_path="get_neighborhoods/houses_with_neighborhoods.csv")

Loading Vancouver road network...
Running fast Dijkstra (single-source)...


In [23]:
# Calculating the address scores based on the distances
from get_address_score import get_address_score
house_with_scores = get_address_score(house_with_distances)

address_scores_result = house_with_scores.sort_values(
    by="Address_Scores", ascending=False)
columns_to_keep = [
    "Address",  "Neighborhood",
    "Driving_Distance_m", "Address_Scores"
]
address_scores_result[columns_to_keep].head(5)

,Address,Neighborhood,Driving_Distance_m,Address_Scores
982,416-418 UNION STREET,Strathcona,1660.572304,100.000000
961,521 UNION STREET,Strathcona,1801.434931,98.506792
492,993 BROUGHTON STREET,West End,1833.572941,98.166115
1107,518 KEEFER STREET,Strathcona,1838.957340,98.109038
1006,601 E PENDER STREET,Strathcona,1891.673695,97.550221


#### Safety Score Calculation

In [24]:
sys.path.append("safety_score")
from safety_score import get_safety_score_data

In [25]:
safety_score_data = get_safety_score_data()
safety_score_data.head(10)

,Neighborhood,Weighted Crime Rate,Min-max Normalized Score,Z-score Normalized Score
Arbutus Ridge,Arbutus Ridge,5384.301982,98,60
Downtown,Downtown,40066.182927,36,15
Dunbar-Southlands,Dunbar-Southlands,4241.601806,100,61
Fairview,Fairview,11764.399879,87,51
Grandview-Woodland,Grandview-Woodland,14814.216426,81,48
Hastings-Sunrise,Hastings-Sunrise,7859.357174,94,57
Kensington-Cedar Cottage,Kensington-Cedar Cottage,7901.428801,93,56
Kerrisdale,Kerrisdale,8732.702406,92,55
Killarney,Killarney,5296.321816,98,60
Kitsilano,Kitsilano,7210.095621,95,57


In [26]:
house_with_scores["Min-max Normalized Score"] = house_with_scores["Neighborhood"].map(safety_score_data.set_index("Neighborhood")["Min-max Normalized Score"])
house_with_scores["Z-score Normalized Score"] = house_with_scores["Neighborhood"].map(safety_score_data.set_index("Neighborhood")["Z-score Normalized Score"])

house_with_scores.head(10)

,Number,Address,List Date,Price,Days on market,Total floor area,Year Built,Age,Lot Size,Latitude,Longitude,geometry,Neighborhood,Driving_Distance_m,Address_Scores,Min-max Normalized Score,Z-score Normalized Score
0,1,3178 GRAVELEY STREET,5/8/2020,1500000,18,2447,1946,74,5674.00,49.270200,-123.037009,POINT (-123.0370093 49.2702),Hastings-Sunrise,6472.196776,48.994603,94,57
1,2,1438 E 28TH AVENUE,1/22/2020,1300000,7,2146,1982,38,3631.98,49.245161,-123.074991,POINT (-123.0749913 49.24516130000001),Kensington-Cedar Cottage,5827.364178,55.830120,93,56
2,3,2831 W 49TH AVENUE,6/18/2019,2650000,1,3108,1929,90,9111.00,49.227603,-123.168387,POINT (-123.1683866 49.2276034),Kerrisdale,8778.309479,24.548764,92,55
3,4,2645 TRIUMPH STREET,6/18/2019,1385000,28,2602,1922,97,4022.70,49.284068,-123.050868,POINT (-123.0508676 49.28406769999999),Hastings-Sunrise,4942.984031,65.204950,94,57
4,5,741-743 E 10TH AVENUE,11/28/2019,1590000,17,1843,1970,49,4026.00,49.261739,-123.088100,POINT (-123.0880999 49.2617389),Mount Pleasant,3754.071589,77.807960,82,48
5,6,86 W 21ST AVENUE,3/3/2020,2565000,7,2439,1989,31,4026.00,49.251692,-123.106888,POINT (-123.106888 49.251692),Riley Park,4039.342348,74.783961,94,57
6,7,2418 E 54TH AVENUE,8/12/2019,1303000,9,2100,1979,40,3298.48,49.219959,-123.057572,POINT (-123.0575719 49.21995889999999),Victoria-Fraserview,9189.303936,20.192037,98,60
7,8,1787 E 57TH AVENUE,6/13/2019,1150000,10,1900,1950,69,6574.82,49.218401,-123.071107,POINT (-123.0711074 49.2184006),Victoria-Fraserview,9029.044772,21.890857,98,60
8,9,2926 TRIMBLE STREET,8/16/2019,2500000,10,2133,2005,14,3360.00,49.260835,-123.205894,POINT (-123.2058942 49.2608353),West Point Grey,8164.768486,31.052577,97,59
9,10,466 E 60TH AVENUE,10/17/2019,2400000,44,3989,2019,0,4412.00,49.215542,-123.095323,POINT (-123.095323 49.2155419),Sunset,8608.146474,26.352569,91,55
